In [28]:
import os
import pandas as pd
from sklearn.metrics import auc
import numpy as np
results_dirpath = '/home/ec2-user/SageMaker/mariano/data/simulations/results/'

models = None
for i in range(1,6):
    path = os.path.join(results_dirpath, f'split_{i}')
    if models is None:
        models = set(os.listdir(path))
    else:
        models = models.intersection(set(os.listdir(path)))
    os.path.exists(path)
    
splits={}
for model in models:
    columns = ['split', 'auc_global_recall', 'auc_test_accuracy', 
               'auc_test_precision', 'auc_test_recall', 'auc_test_f1',
               'avg_prediction_time', 'avg_training_time',
              ]
    averages = {}
    for column in columns:
        averages[column]=[]
        
    for i in range(1,6):
        averages['split'].append(i)
        df = pd.read_csv(os.path.join(results_dirpath, f'split_{i}', f'{model}'), index_col=0)
        x = df['loop_no']
        df['global_recall'] = df['relevant_count']/df['relevant_count'].iloc[-1]
        df = df.drop(labels=['test_TP'],axis=1)
        for column in ['test_accuracy', 'test_precision', 'test_recall', 'test_f1', 'global_recall']:
            y = df[column]
            auc_score = auc(x,y)
            averages['auc_'+column].append(auc_score)
        for column in ['prediction_time', 'training_time']:
            average_value = np.average(df[column])
            averages['avg_'+column].append(average_value)
            
#         print()
        splits[f'split_{i}']=df 
    
    averages['split'].append('average')
    for column in ['test_accuracy', 'test_precision', 'test_recall', 'test_f1', 'global_recall']:
        averages['auc_'+column].append(np.average(averages['auc_'+column]))
    for column in ['prediction_time', 'training_time']:
        average_value = np.average(df[column])
        averages['avg_'+column].append(average_value)
       
    averages_df = pd.DataFrame(averages)
    averages_df_file =  os.path.join(results_dirpath,'averages',f'{model}')
    if os.path.isfile(averages_df_file):
        print(f'[WARNING] model ./averages/{model} is being overwritten')
    averages_df.to_csv(averages_df_file)

[WARNING] model ./averages/random_ranking_logreg_bow.csv is being overwritten
[WARNING] model ./averages/MLP_with_GloVe.csv is being overwritten
[WARNING] model ./averages/all_three_models.csv is being overwritten
[WARNING] model ./averages/highlighter_only_bow.csv is being overwritten
[WARNING] model ./averages/MLP_with_BOW.csv is being overwritten


In [27]:
import os
import pandas as pd
from sklearn.metrics import auc
import numpy as np
results_dirpath = '/home/ec2-user/SageMaker/mariano/data/simulations/results/'

models = None
for i in range(1,6):
    path = os.path.join(results_dirpath, f'split_{i}')
    if models is None:
        models = set(os.listdir(path))
    else:
        models = models.intersection(set(os.listdir(path)))
    os.path.exists(path)
    
splits={}
for model in models:

    values = None
    for i in range(1,6):
        df = pd.read_csv(os.path.join(results_dirpath, f'split_{i}', f'{model}'), index_col=0)
        if values is None:
            values = df.values
            values = values[:,:,np.newaxis]
        else:
            to_concat = df.values
            if to_concat.shape[0]>values.shape[0]:
                to_concat = to_concat[:values.shape[0],:]
            values = np.append(values, values=to_concat[:,:,np.newaxis], axis=2)

    

    values = np.average(values,axis=2)
    new_df = pd.DataFrame(values,columns=df.columns)
#     averages_df = None # pd.DataFrame(averages)
    
    averages_df_file =  os.path.join(results_dirpath,'averages',f'avg_auc_{model}')
    if os.path.isfile(averages_df_file):
        print(f"[WARNING ] model ./averages/" + averages_df_file.split('/')[-1] + "is being overwriten.")
    else:
        print(f"[CREATING] model ./averages/" + averages_df_file.split('/')[-1])
        new_df.to_csv(averages_df_file)

[WARNING ] model ./averages/avg_auc_random_ranking_logreg_bow.csvis being overwriten.
[WARNING ] model ./averages/avg_auc_MLP_with_GloVe.csvis being overwriten.
[WARNING ] model ./averages/avg_auc_all_three_models.csvis being overwriten.
[WARNING ] model ./averages/avg_auc_highlighter_only_bow.csvis being overwriten.
[CREATING] model ./averages/avg_auc_LogReg_BoW_uncertanty_ranking.csv
[WARNING ] model ./averages/avg_auc_MLP_with_BOW.csvis being overwriten.


In [14]:
new_df

,loop_no,relevant_count,irrelevant_count,prediction_time,training_time,train_accuracy,train_precision,train_recall,train_f1,test_accuracy,test_precision,test_recall,test_f1,test_TP,unlabeled(train)_TP
0,0.0,5.0,0.0,0.000401,0.051373,1.0,1.0,1.0,1.0,0.929001,0.692828,0.377919,0.479672,49.8,195.4
1,1.0,12.0,3.0,0.000399,0.031640,1.0,1.0,1.0,1.0,0.091184,0.089583,0.995354,0.164132,130.2,509.2
2,2.0,12.8,12.2,0.000399,0.023446,1.0,1.0,1.0,1.0,0.939165,0.652227,0.763875,0.679683,101.6,381.0
3,3.0,19.2,15.8,0.000399,0.017019,1.0,1.0,1.0,1.0,0.930791,0.624127,0.751105,0.647542,99.6,380.4
4,4.0,25.8,19.2,0.000400,0.015362,1.0,1.0,1.0,1.0,0.932434,0.593593,0.829219,0.684259,109.8,402.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,577.0,522.8,5252.2,0.000407,0.002648,1.0,1.0,1.0,1.0,0.956605,0.758315,0.757523,0.756742,99.4,0.0
578,578.0,522.8,5262.2,0.000402,0.003278,1.0,1.0,1.0,1.0,0.956880,0.759119,0.759004,0.758119,99.6,0.0
579,579.0,522.8,5272.2,0.000419,0.002693,1.0,1.0,1.0,1.0,0.957017,0.758471,0.763706,0.760324,100.2,0.0
580,580.0,522.8,5282.2,0.000400,0.002725,1.0,1.0,1.0,1.0,0.956468,0.760884,0.751082,0.754740,98.6,0.0


In [12]:
for key in averages.keys():
    print(f'{key:12}: {len(averages[key])}')

split       : 6
auc_global_recall: 6
auc_test_accuracy: 6
auc_test_precision: 6
auc_test_recall: 6
auc_test_f1 : 6
avg_prediction_time: 5
avg_training_time: 5


In [4]:
df

,loop_no,relevant_count,irrelevant_count,prediction_time,training_time,train_accuracy,train_precision,train_recall,train_f1,test_accuracy,test_precision,test_recall,test_f1,unlabeled(train)_TP,global_recall
0,0,5,0,0.000395,0.002456,0.000000,0.000000,0.000000,0.000000,0.924502,0.000000,0.000000,0.000000,0,0.009191
1,1,6,9,0.000396,0.002645,1.000000,1.000000,1.000000,1.000000,0.945093,0.741935,0.418182,0.534884,202,0.011029
2,2,16,9,0.000395,0.001793,0.920000,0.888889,1.000000,0.941176,0.603294,0.159884,1.000000,0.275689,528,0.029412
3,3,17,18,0.000393,0.001353,1.000000,1.000000,1.000000,1.000000,0.934111,0.535354,0.963636,0.688312,503,0.031250
4,4,22,23,0.000438,0.001105,0.977778,0.956522,1.000000,0.977778,0.945779,0.596273,0.872727,0.708487,455,0.040441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,577,544,5231,0.000393,0.000523,0.973680,0.903292,0.806985,0.852427,0.962938,0.750000,0.763636,0.756757,0,1.000000
578,578,544,5241,0.000397,0.000537,0.973725,0.903292,0.806985,0.852427,0.962938,0.750000,0.763636,0.756757,0,1.000000
579,579,544,5251,0.000410,0.000519,0.973770,0.903292,0.806985,0.852427,0.962938,0.750000,0.763636,0.756757,0,1.000000
580,580,544,5261,0.000394,0.000510,0.973816,0.903292,0.806985,0.852427,0.962938,0.750000,0.763636,0.756757,0,1.000000


In [51]:
pd.DataFrame(averages)

,split,auc_global_recall,auc_test_accuracy,auc_test_precision,auc_test_recall,auc_test_f1
0,1,525.409007,559.228552,423.066937,477.345455,447.072881
1,2,525.409007,559.228552,423.066937,477.345455,447.072881
2,3,525.409007,559.228552,423.066937,477.345455,447.072881
3,4,525.409007,559.228552,423.066937,477.345455,447.072881
4,5,525.409007,559.228552,423.066937,477.345455,447.072881
5,average,525.409007,559.228552,423.066937,477.345455,447.072881


In [43]:
df.average?

Object `df.average` not found.
